# LPP implementatie

De `ulpp` module verzorgt de codering en decodering van berichten in het binaire Cayenne LPP formaat.
Voor een beschrijving van dit formaat, zie:

* https://docs.mydevices.com/docs/lorawan/cayenne-lpp
* https://github.com/myDevicesIoT/CayenneLPP
* https://github.com/smlng/pycayennelpp
    * dit is een uitgebreide Python-implementatie van Cayenne LPP
    * deze beschrijft meer sensor-formaten dan de oorspronkelijk LPP versie

De `lppjson` module bevat een enkele functie, `lpp_to_json`, om binaire LPP berichten om te zetten in JSON string-formaat. Deze wordt o.a. gebruikt door de radio-logger.

De `ulpp` module is een onafhankelijke implementatie van de codering en decodering, bedoeld om het gebruik van LPP voor de microbit eenvoudig te houden.

De belangrijke elementen van deze library zijn:

* class LppBuffer
    * `LppBuffer(prefix, maxsize)`
    * `buffer.add_digital_input(channel, value)` - etc.
    * met varianten voor elk type sensor 
    * `frame.to_bytes() -> bytes`
* `bytes_to_dict(bytes)`
* `dict_to_bytes(dict)`

Via `LppBuffer` en `buffer.add_xxsensor(channel, value)` bouw je een array van getallen op die je als bytes-waarde kunt versturen.

Deze bytes-waarde kun je ook omzetten (decoderen) met `bytes_to_dict()` in een Python dictionary - die je vervolgens kunt omzetten in JSON, voor een MQTT bericht.
Via de codering `dict_to_bytes()` kun je een Python dictionary (in de geschikte LPP vorm!) omzetten naar een LPP bytes-waarde.

* `dict_to_bytes()` verwerkt in deze implementatie (voorlopig) alleen actuator-data: deze is bedoeld voor downlink-berichten.

Daarnaast is er nog een afzonderlijke functie:

* `json_to_lpp(s: str) -> bytes`

Voorbeelden van het gebruik:

**IoT-knoop: opbouwen van een reeks sensorwaarden**, en versturen als bytes-waarde (uplink).

```Python
prefix = bytes([1,2,3]) # prefix before LPP data
lpp = LppBuffer(data=prefix, maxsize=60)
lpp.add_digital_input(0, 1)
lpp.add_digital_output(1, 1)
lpp.add_analog_input(2, 1234)
lpp.add_analog_output(4, -1234)
lpp.add_luminosity(7, 345)
lpp.add_presence(8, 1)
lpp.add_barometer(9, 10230)
lpp.add_temperature(5, int(23.4*10))
lpp.add_digital_input(10, 1)
radio.send_bytes(lpp.to_bytes())
```

**IoT-knoop: omzetten van een ontvangen (binair) LPP-bericht in acties** - afhandelen van actuator-data.

(Dit wordt voorlopig ad-hoc in de IoT-knoop opgelost.)

**Gateway: Omzetten (decoderen) van een ontvangen byte-array** in een JSON-string, om te versturen via MQTT. (Uplink-berichten in de gateway.)

```Python
# data bevat ontvangen radio-bericht: header, lpp payload
nodeID = data[1] * 256 + data[2]
counter = data[3] * 256 + data[4]
lpp_dict = bytes_to_dict(data[5:])
msg = {'nodeid': '{0:x}'.format(nodeID),  # in hex format
       'counter': counter,
       'payload': lpp_dict
       }
mqtt.publish(topic, json.dumps(msg))
```

**Gateway: van ontvangen JSON-string naar binair LPP downlink-bericht** 

```Python
obj = json.loads(msg)
lpp_bytes = dict_to_bytes(obj)   # from JSON string to LPP byte-array
header_bytes = bytes([0x0B, nodeID // 256, nodeID % 256, 0, 0]) # downlink header  
radio.send_bytes(header_bytes + lpp_bytes)    # send header followed by LPP data
```

**Radio-logger: omzetten van (binaire) LPP-bytes naar LPP in JSON:** `lpp_to_json(data: bytes) -> str`.

```Python
print(lpp_to_json(msg[5:]) # skip radio msg header
```

(module-ulpp)=
## module `ulpp`

In [1]:
class LppBuffer(object):

    # some assumptions:
    # - the value-parameters are in the LPP-required format and range
    #   i.e. scaling is done by the caller (if needed)
    #   all value-parameters are int
    
    def __init__(self, data=b'', maxsize=32):
        self.buffer = bytearray(data)
        self.maxsize = maxsize
        self.pos = 0

    def __str__(self):
        return str(list(self.buffer))
    
    def to_bytes(self) -> bytes:
        return bytes(self.buffer)
    
    def add_byte(self, channel, tag, value):
        if len(self.buffer) + 3 > self.maxsize:
            raise OverflowError
        self.buffer.append(channel)
        self.buffer.append(tag)
        self.buffer.append(value)

    def add_unsigned_int16(self, channel, tag, value):
        if len(self.buffer) + 4 > self.maxsize:
            raise OverflowError        
        self.buffer.append(channel)
        self.buffer.append(tag)
        if value >= 65536:
            value = value % 65536
        (hi, lo) = divmod(value, 256)
        self.buffer.append(hi)
        self.buffer.append(lo)

    def add_signed_int16(self, channel, tag, value):
        while value < 0:
            value = value + 65536
        self.add_unsigned_int16(channel, tag, value)

    def add_digital_input(self, channel, value):
        self.add_byte(channel, 0, value)

    def add_digital_output(self, channel, value):
        self.add_byte(channel, 1, value)
        
    def add_analog_input(self, channel, value):
        self.add_signed_int16(channel, 2, value)

    def add_analog_output(self, channel, value):
        self.add_signed_int16(channel, 3, value)

    def add_luminosity(self, channel, value):
        self.add_unsigned_int16(channel, 101, value)

    def add_presence(self, channel, value):
        self.add_byte(channel, 102, value)

    def add_temperature(self, channel, value):
        # temperature: 0.1C, signed int
        self.add_signed_int16(channel, 103, value)

    def add_humidity(self, channel, value):
        # rel. humidity: 0.5% unsigned byte
        self.add_byte(channel, 104, value)

    def add_barometer(self, channel, value):
        # barometric pressue: 0.1 hPa unsigned int16
        self.add_unsigned_int16(channel, 115, value)

def bytes_to_dict(data: bytes) -> dict:

    pos = 0

    def nextbyte():
        nonlocal pos
        if pos >= len(data):
            raise OverflowError
        value = data[pos]
        pos = pos + 1
        return value
    
    def nextunsignedint():
        hi = nextbyte()
        lo = nextbyte()
        value = hi * 256 + lo
        return value 
    
    def nextint():
        value = nextunsignedint()
        if value > 32767:
            value = value - 65536
        return value

    pos = 0
    obj = {}

    while pos < len(data):
        channel = nextbyte()
        tag = nextbyte()
        if tag == 0:
            value = nextbyte()
            obj[channel] = {'dIn': value}
        elif tag == 1:
            value = nextbyte()
            obj[channel] = {'dOut': value}                
        elif tag == 2:
            value = nextint()
            obj[channel] = {'aIn': value}                
        elif tag == 3:
            value = nextint()
            obj[channel] = {'aOut': value}
        elif tag == 101:
            value = nextunsignedint()
            obj[channel] = {'luminosity': value}                
        elif tag == 102:
            value = nextbyte()
            obj[channel] = {'presence': value}                
        elif tag == 103:
            value = nextint()                
            obj[channel] = {'temperature': value}                
        elif tag == 104:
            value = nextbyte()
            obj[channel] = {'humidity': value}                
        elif tag == 115:
            value = nextunsignedint()
            obj[channel] = {'barometer': value}
            
    return obj

def dict_to_bytes (obj: dict) -> bytes:
    lpp = LppBuffer()

    for channel in obj:
        item = obj[channel]  # item is an object with a single key...
        if type(channel) is str:
            channel = int(channel)
        for key in item:
            if key == 'dOut':
                lpp.add_digital_output(channel,  item[key])
            elif key == 'aOut':
                lpp.add_analog_output(channel,  item[key])
            else:
                # not implemented, raise exception? only output allowed
                raise ValueError('only output values allowed in actuator msg')

    return lpp.to_bytes()

---

(module-lppjson)=
## module `lppjson`

De module `lppjson` heeft 1 functie, `lpp_to_json`. Deze wordt gebruikt in de radio-logger.

In [2]:
def lpp_to_json(data: bytes) -> str:
    
    pos = 0
    
    def nextbyte():
        nonlocal pos
        if pos >= len(data):
            raise OverflowError
        value = data[pos]
        pos = pos + 1
        return value
    
    def nextunsignedint():
        hi = nextbyte()
        lo = nextbyte()
        return hi * 256 + lo 
    
    def nextint():
        value = nextunsignedint()
        if value > 32767:
            value = value - 65536
        return value
    
    pos = 0
    json_string = '{'
    sep = ''

    while pos < len(data):
        channel = nextbyte()
        tag = nextbyte()
        value = 0
        tagname = ''
        if tag == 0:
            value = nextbyte()
            tagname = 'dIn'
        elif tag == 1:
            value = nextbyte()
            tagname = 'dOut'
        elif tag == 2:
            value = nextint()
            tagname = 'aIn'
        elif tag == 3:
            value = nextint()
            tagname = 'aOut'
        elif tag == 101:
            value = nextunsignedint()
            tagname = 'luminosity'
        elif tag == 102:
            value = nextbyte()
            tagname = 'presence'
        elif tag == 103:
            value = nextint()                
            tagname = 'temperature'
        elif tag == 104:
            value = nextbyte()
            tagname = 'humidity'
        elif tag == 115:
            value = nextunsignedint()
            tagname = 'barometer'
        json_string += '{0}"{1}": {{"{2}": {3}}}'.format(sep, channel, tagname, value)
        sep = ', '
        
    json_string += '}' 
    return json_string

## functie `json_to_bytes`

In [3]:
def json_to_bytes (js: str) -> bytes:
    obj = json.loads(js)
    return dict_to_bytes(obj)

---

## Testen

De onderstaande testen kunnen uitgevoerd worden in Jupyter Notebook.

### test: opbouwen van een binaire LPP-waarde

Deze code vind je in de IoT-knoop, bij het versturen van de sensorwaarden.
En mogelijk bij het opbouwen van een LPP-waarde met actuator-waarden, vanuit een toepassing.

In [4]:
header = bytes([1,2,3,4,5])
frame = LppBuffer(data=header, maxsize=60)
frame.add_digital_input(0, 1)
frame.add_digital_output(1, 1)
frame.add_analog_input(2, 1234)
frame.add_analog_output(4, -1234)
frame.add_luminosity(7, 345)
frame.add_presence(8, 1)
frame.add_barometer(9, 10230)
frame.add_temperature(5, int(23.4*10))
frame.add_digital_input(10, 1)
  # radio.send_bytes(frame.to_bytes())
buffer = frame.to_bytes()
print(list(buffer))

[1, 2, 3, 4, 5, 0, 0, 1, 1, 1, 1, 2, 2, 4, 210, 4, 3, 251, 46, 7, 101, 1, 89, 8, 102, 1, 9, 115, 39, 246, 5, 103, 0, 234, 10, 0, 1]


### van LPP-bytes naar JSON

In de gateway worden ontvangen (binaire) sensor-berichten omgezet in JSON-formaat, om te versturen via MQTT.

In [5]:
import json

In [16]:
lppdata = bytes_to_dict(buffer[5:]) # skip header
lppdata

{0: {'dIn': 1},
 1: {'dOut': 1},
 2: {'aIn': 1234},
 4: {'aOut': -1234},
 7: {'luminosity': 345},
 8: {'presence': 1},
 9: {'barometer': 10230},
 5: {'temperature': 234},
 10: {'dIn': 1}}

In [7]:
jsondata = json.dumps(lppdata)
jsondata

'{"0": {"dIn": 1}, "1": {"dOut": 1}, "2": {"aIn": 1234}, "4": {"aOut": -1234}, "7": {"luminosity": 345}, "8": {"presence": 1}, "9": {"barometer": 10230}, "5": {"temperature": 234}, "10": {"dIn": 1}}'

### Van JSON naar LPP-bytes

In [8]:
lppdata1 = json.loads(jsondata)
lppdata1

{'0': {'dIn': 1},
 '1': {'dOut': 1},
 '2': {'aIn': 1234},
 '4': {'aOut': -1234},
 '7': {'luminosity': 345},
 '8': {'presence': 1},
 '9': {'barometer': 10230},
 '5': {'temperature': 234},
 '10': {'dIn': 1}}

In [9]:
lppbytes1 = dict_to_bytes(lppdata1)
lppbytes1

ValueError: only output values allowed in actuator msg

Bovenstaande werkt niet in de (beperkte) implementatie van dict_to_bytes. Dit werkt wel als we de dictionary beperken tot alleen actuator-waarden.

In [10]:
lppbytes1 = dict_to_bytes({'1': {'dOut': 1}, '4': {'aOut': -1234}})
list(lppbytes1)

[1, 1, 1, 4, 3, 251, 46]

Nog een extra voorbeeld, met alleen actuator-waarden.

In [11]:
lpp = LppBuffer()
lpp.add_analog_output(3, 123)
lpp.add_digital_output(5, 1)
lpp.add_analog_output(6, -12)

lppbuf = lpp.to_bytes()

lppdict = bytes_to_dict(lppbuf)

In [12]:
list(dict_to_bytes(lppdict))

[3, 3, 0, 123, 5, 1, 1, 6, 3, 255, 244]

**Opmerking.** Het is ook mogelijk om in de microbit de codering en codering uit te schrijven, als we niet genoeg geheugen hebben voor zo'n library. Zo complex is dat nu ook weer niet. Voor de microbit V2 is het geheugen niet zo'n probleem, maar voor de V1 (in Python) wel.

(In de RF69-versie is die codering en codering ook uitgeschreven, zonder gebruik van een library.)

### Test `lpp_to_json`

In [13]:
lpp_to_json([1, 1, 1, 4, 3, 251, 46])

'{"1": {"dOut": 1}, "4": {"aOut": -1234}}'

In [14]:
lpp_to_json([3, 0, 0, 2, 0, 0, 4, 2, 0, 0, 6, 103, 0, 200])

'{"3": {"dIn": 0}, "2": {"dIn": 0}, "4": {"aIn": 0}, "6": {"temperature": 200}}'

In [15]:
lpp_to_json([])

'{}'

Het omgekeerde: JSON naar LPP, gaat in twee stappen: (i) JSON naar Python object (via de json loads); (ii) dict_to_lpp - maar dit is beperkt tot een aantal actuator-waarden, specifiek voor de gateway downlink.